In [0]:
rawdf1=spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/custs",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.createOrReplaceTempView("v1")
spark.sql("select * from v1 where age >50").createOrReplaceTempView("_sqldf")

spark.sql("create or replace temp view v2 as select * from v1 where age <50")




In [0]:
%sql
select * from v2

In [0]:
rawdf1.printSchema()
print(rawdf1.columns)
rawdf1.dtypes
print(rawdf1.schema)

In [0]:
%sql
describe v1

In [0]:
rawdf1.createOrReplaceTempView("rawdf1view")
print("actual count of the data","select count(*) from rawdf1view")
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

In [0]:
%sql
SELECT COUNT(*) AS dedup_all_columns_count 
FROM (SELECT distinct * FROM rawdf1view) t;

In [0]:
%sql
SELECT COUNT(*) AS dedup_all_columns_count 
FROM (SELECT id FROM rawdf1view where id is not null) t;

In [0]:
from pyspark.sql.session import SparkSession#15lakhs
spark=SparkSession.builder.appName("WD36 - ETL Pipeline - Bread & Butter").getOrCreate()#

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified")
#2. Multiple files (with same or different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/custsmodified","/Volumes/lakehouse1/dbread/read_volume/custsmodified"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/","/Volumes/workspace/wd36schema/ingestion_volume/staging/"],recursiveFileLookup=True,pathGlobFilter="custsm*")

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)
print(cleandf1.count())#count will return the original count of the raw data
print(len(cleandf1.collect()))#collect+len will return the dropmalformed count of the raw data

In [0]:
struttype1=StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#

In [0]:
#Creating rejection dataset to send to our source system for future fix
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True),StructField("corruptedrows",StringType())])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#Create a reject dataset
rejectdf1=cleandf1.where("corruptedrows is not null")
#display(rejectdf1)
rejectdf1.write.csv("/Volumes/lakehouse1/dbread/read_volume/Rejected/",mode="overwrite",header=True)
retaineddf1=cleandf1.where("corruptedrows is null")
print("Overall rows in the source data is ",len(cleandf1.collect()))
print("Rejected rows in the source data is ",len(rejectdf1.collect()))
print("Clean rows in the source data is ",len(retaineddf1.collect()))

In [0]:
cleanseddf=cleandf1.filter("corruptedrows is null").drop("corruptedrows")
cleanseddf.createOrReplaceTempView("temp")
# spark.sql("select *, count(*) cnt from temp group by 1,2,3,4,5 having cnt>1").show()
#display(cleanseddf)
print("na.drop",cleanseddf.na.drop(how='any').count())
print("distinct",cleanseddf.distinct().count())
#cleanseddf.na.drop(how='any',subset=['id','lastname']).show()

In [0]:
display(cleanseddf)

In [0]:
find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
cleanseddf \
    .na.fill("Not Provided", subset=["firstname", "lastname", "profession"]) \
    .na.fill(-1, subset=["age"]) \
    .na.replace(find_replace_values_dict1,subset=["profession"]).show()


In [0]:
cleanseddf.createOrReplaceTempView("temp1")

spark.sql("create or replace temp view temp2 as select id, firstname, coalesce(lastname,'n/a') lastname, age, coalesce(profession,'n/a') profession from temp1").show()
#

In [0]:
cleanseddf=rawdf1.na.drop(how="any")

In [0]:
scrubbeddf1=cleanseddf.na.fill('not provided',subset=["lastname","profession"])#fill will help us replace nulls with some value
display(scrubbeddf1)
find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
find_replace_values_dict2={'not provided':'NA'}
scrubbeddf2=scrubbeddf1.na.replace(find_replace_values_dict1,subset=["profession"])#fill function is helping us find and replace the values
scrubbeddf3=scrubbeddf2.na.replace(find_replace_values_dict2,subset=["lastname"])
display(scrubbeddf3)

In [0]:
display(scrubbeddf3.where("id in ('4000001')"))#before row level dedup
dedupdf1=scrubbeddf3.distinct()#It will remove the row level duplicates
display(dedupdf1.where("id in ('4000001')"))

In [0]:
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))

In [0]:
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#dedupdf1.coalesce(1).where("id in ('4000003')").orderBy(["id","age"],ascending=[True,False]).show(3)
dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))

In [0]:
cleandf1.createOrReplaceTempView("cleandf1")


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW scrubbeddf1 AS
SELECT
  id,
  firstname,
  COALESCE(lastname, 'not provided')  AS lastname,
  age,
  COALESCE(profession, 'not provided') AS profession
FROM cleandf1;

In [0]:
scrubbeddf1.createOrReplaceTempView("scrubbeddf1")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW scrubbeddf2 AS
SELECT
  id,
  firstname,
  lastname,
  age,
  CASE
    WHEN profession = 'Pilot' THEN 'Captain'
    WHEN profession = 'Actor' THEN 'Celeberity'
    ELSE profession
  END AS profession
FROM scrubbeddf1;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW dedupdf2_priority AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY id
           ORDER BY id,age DESC
         ) AS rn
  FROM scrubbeddf2
) t
WHERE rn = 1;
select * from dedupdf2_priority;

In [0]:
from pyspark.sql.functions import lit,initcap,col
#withColumn("stringcolumnname to add in the df",lit('hardcoded')/initcap(col("colname")))
standarddf1=dedupdf2.withColumn("sourcesystem",lit("Retail"))#SparkSQL - DSL(FBP)
display(standarddf1.limit(20))

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW duplicateview AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY id ORDER BY id,age DESC) AS rn
  FROM scrubbeddf2
) t;
--select * from duplicateview where rn=1;--deduplicated data
select * from duplicateview qualify (ROW_NUMBER() OVER (PARTITION BY id ORDER BY id,age DESC))=1;

-- select id, count(*)cnt from duplicateview group by id having cnt>1 order by id;--

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW standarddf1 AS
SELECT
  *,
  'Retail' AS sourcesystem
FROM dedupdf2_priority 
where upper(id) =lower(id);--wanted to take only number values in id column

select * from standarddf1 limit 10;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW standarddf2 AS
SELECT
  id,
  firstname,
  lastname,
  age,
  INITCAP(profession) AS profession,
  sourcesystem
FROM standarddf1;

select * from standarddf2 limit 10;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW standarddf3 AS
SELECT
  CASE
    WHEN cast(id as string) = 'nine'  THEN '9'
    WHEN cast(id as string) = 'ten'   THEN '10'
    ELSE id
  END AS id,
  firstname,
  lastname,
  REGEXP_REPLACE(age, '-', '') AS age,
  profession,
  sourcesystem
FROM standarddf2;

select * from standarddf3 limit 10;

In [0]:
%sql
create or replace temp view standarddf4 as 
SELECT 
    CAST(id AS BIGINT) AS id,
    firstname,
  lastname,
  CAST(age AS SMALLINT) AS age,
  profession,
  sourcesystem
FROM standarddf3;

In [0]:
%sql
create or replace temp view standarddf5 as
select 
id custid,
firstname,
lastname,
age,
profession,
sourcesystem as srcsystem
 from standarddf4;

 select * from standarddf5;

In [0]:
%sql
create or replace temp view mungeddf as 
SELECT 
    custid, 
    age, 
    firstname,
    lastname,
    profession,
    srcsystem
FROM standarddf5;

SELECT * FROM mungeddf LIMIT 10;

In [0]:
original_filename='custsmodified_25/30/12.csv'
derived_datadt=original_filename.split('_')[1].split('.')[0]
spark.sql(f"""
    create or replace temp view enrichdf1 as 
    SELECT 
        *,
        '{derived_datadt}' AS datadt,
        current_date() AS loaddt
    FROM mungeddf
""")


In [0]:
%sql
select * from enrichdf1 limit 10;

In [0]:
%sql
create or replace temp view enrichdf2 as 
SELECT 
    *,
    substring(profession, 1, 1) AS professionflag
FROM enrichdf1;

SELECT * FROM enrichdf2 LIMIT 20;

In [0]:
%sql
create or replace temp view enrichdf3 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    profession,
    srcsystem AS sourcename, -- Renaming occurs here
    datadt,
    loaddt,
    professionflag AS profflag -- Renaming occurs here
FROM enrichdf2;

SELECT * FROM enrichdf3 LIMIT 20;

In [0]:
%sql
create or replace temp view enrichdf4 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    concat(profession, '-', profflag) AS profession, -- Overwrites/Modifies the profession column
    sourcename,
    datadt,
    loaddt,
    profflag
FROM enrichdf3;

SELECT * FROM enrichdf4 LIMIT 20;